In [ ]:
import os
import json
from PIL import Image, ImageOps, ImageDraw
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from mpl_toolkits.axes_grid1 import ImageGrid
from datetime import datetime, timedelta
import numpy.ma as ma
from datetime import datetime
import pandas as pd

In [ ]:
data_dir='/home/ajai-krishna/work/Phenocam_d3/Phenocamdata'
plot_dir='/home/ajai-krishna/work/Phenocam_d3/Plots'
file_path=[os.path.join(data_dir,i) for i in os.listdir(data_dir)]
file_path

In [ ]:
df1 = pd.DataFrame(file_path, columns=['file_path'])
df1['image_type'] = df1['file_path'].apply(lambda x: 'rgb' if x.lower().endswith('_color.jpg') 
                                           else ('ir' if x.lower().endswith('_ir.jpg') else ('NDVI' if x.lower().endswith('_ndvi.jpg') else 'other')))
df1['date_time'] = df1['file_path'].apply(lambda x: os.path.basename(x).split('APU_pos_01_')[-1].rsplit('_', 1)[0])
#df1['datetime_obj'] = pd.to_datetime(df1['date_time'], format='%Y_%m_%d_%H%M%S')
df1['date'] = df1['date_time'].apply(lambda x: "_".join(x.split('_')[0:3]))
df1['time'] = df1['date_time'].apply(lambda x: "".join(x.split('_')[3:][:2]))
#df1['time'] = pd.to_datetime(df1['time'], format='%H%M').dt.time
df1

In [ ]:
df_ndvi=df1[df1['image_type']=='NDVI']


In [ ]:
ndvi_img=np.array(Image.open(df_ndvi['file_path'][3]))
rows=cols=10
h, w = ndvi_img.shape[:2]
tile_h= h // rows
tile_w = w//rows
tiles = []
mean_ndvi = np.zeros((rows, cols))

for i in range(rows):
    for j in range(cols):
        tile = ndvi_img[
            i*tile_h:(i+1)*tile_h,
            j*tile_w:(j+1)*tile_w
        ]
        tiles.append(tile)
        mean_ndvi[i,j]=np.nanmean(tile)

In [ ]:
tiles[0].shape


In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid

fig = plt.figure(figsize=(8,8))
grid = ImageGrid(fig, 111, nrows_ncols=(10, 10), axes_pad=0.02)

k = 0
for i in range(10):
    for j in range(10):
        tile = ndvi_img[
            i*tile_h:(i+1)*tile_h,
            j*tile_w:(j+1)*tile_w
        ]
        grid[k].imshow(tile)
        grid[k].axis('off')
        k += 1

plt.show()


In [ ]:
dates = df1['date'].unique()
selected_fils = []
for date in dates:
    group = df1[df1['date'] == date]
    ir_times = group[group['image_type'] == 'ir']['time'].values
    rgb_times = group[group['image_type'] == 'rgb']['time'].values
    common_times = set(ir_times).intersection(set(rgb_times))
    time1 = common_times.pop() if common_times else None
    if time1:
        ir_file = group[(group['image_type'] == 'ir') & (group['time'] == time1)]['file_path'].values[0]
        rgb_file = group[(group['image_type'] == 'rgb') & (group['time'] == time1)]['file_path'].values[0]
        selected_fils.append({'date': date, 
                              'time': time1, 
                              'ir_file':ir_file, 
                              'rgb_file':rgb_file})


In [ ]:
df=pd.DataFrame(selected_fils).sort_values(by=['date','time']).reset_index(drop=True)
df

In [ ]:

def select_time(group):
    ir_time = group[group['image_type'] == 'ir']['time'].values
    rgb_df = group[group['image_type'] == 'rgb']['time'].values
    for time1 in ir_time:
        if time1 in rgb_df:
            return time1    

df1['temp']= df1.groupby(['date']).apply(select_time)

In [ ]:
df1['temp'].unique()
df1

In [ ]:
def ndvi_calculation(ir_path, rgb_path):
    # Load images
    ir_img = np.array(Image.open(ir_path)).astype(float)
    rgb = np.array(Image.open(rgb_path)).astype(float)

    # Extract IR band
    if ir_img.ndim == 3:
        ir = ir_img[:, :, 0]   # take one channel
    else:
        ir = ir_img

    # Extract Red band
    red = rgb[:, :, 0]

    # Avoid division by zero
    denominator = ir + red
    denominator[denominator == 0] = np.nan

    # NDVI calculation
    ndvi = (ir - red) / denominator

    # Return mean NDVI
    return np.nanmean(ndvi)



In [ ]:
df['ndvi'] = df.apply(
    lambda row: ndvi_calculation(row['ir_file'], row['rgb_file']),
    axis=1
)


In [ ]:


fig, ax = plt.subplots(figsize=(12,4))

ax.plot(df['date'], df['ndvi'], marker='o')

ax.set_ylabel('NDVI')
ax.set_xlabel('Date')
ax.set_title('NDVI Time Series')
# ax.set_ylim(-1, 1)
ax.grid(True)

# Auto-format dates
fig.autofmt_xdate()
plt.xticks(rotation=90, ha='right')
plot_path=os.path.join(plot_dir,'ndvi_plot')
plt.savefig(plot_path+'.png', bbox_inches='tight', dpi=300)
plt.show()
# os.path.join(plt.savefig('ndvi.json'))


In [ ]:
plot_data= {
    "title": "NDVI Time Series",
    "xlabel": "Date",
    "ylabel": "NDVI",
    "x": df["date"].astype(str).tolist(), 
    "y": df["ndvi"].tolist(),
    "marker": "o",
    "grid": True
}

In [ ]:
json_path = os.path.join(plot_dir, "ndvi_plot.json")

with open(json_path, "w") as f:
    json.dump(plot_data, f, indent=4)

print(f"Saved NDVI plot data to: {json_path}")


In [ ]:

def select_data(group):
    ir_image_path = group[group['image_type'] == 'ir']['file_path'].values
    rgb_image_path = group[group['image_type'] == 'rgb']['file_path'].values
    ir_time=group[group['image_type'] == 'ir']['time'].values
    
df1.groupby('date').apply(select_data)

In [ ]:
test_image_ir='/home/ajai-krishna/work/Phenocam_d3/Phenocamdata/APU_pos_01_2025_07_28_13_44_47_ir(1).jpg'
# test_image_color='/home/ajai-krishna/work/Phenocam_d3/Phenocamdata/APU_pos_01_2025_07_28_13_42_14(1).jpg'
test_image_color='/home/ajai-krishna/work/Phenocam_d3/Phenocamdata/APU_pos_01_2025_07_28_13_44_47_color(1).jpg'
img_ir= Image.open(test_image_ir)   
img_color=Image.open(test_image_color)
img_ir.load()
img_color.load()
# img
#img_ir = np.mean(img_ir,axis=3

In [ ]:
mean_ir = np.mean(np.array(img_ir).astype(float),axis=2)
mean_vis = np.mean(np.array(img_color).astype(float),axis=2)

plt.hist(mean_vis)

In [ ]:
ndvi_vis = (mean_ir - mean_vis)/(mean_vis+mean_ir)

In [ ]:
# Convert images to grayscale (returns new images)
# img_ir = img_ir.convert('L')
# img_color = img_color.convert('RGB')  # Keep color as RGB to extract red channel

# Convert to numpy arrays
img_ir_array = np.array(img_ir)
img_color_array = np.array(img_color)

# Convert to float for calculation
img_ir = img_ir_array[:, :, 0].astype(float)
img_r = img_color_array[:, :, 0].astype(float)

In [ ]:


# Calculate NDVI with division by zero handling
denominator = img_ir + img_r
ndvi = np.where(denominator != 0, (img_ir - img_r) / denominator, 0)

# Display NDVI
plt.imshow(ndvi, cmap='RdYlGn',vmin=-1)  # Use appropriate colormap for NDVI
plt.colorbar()
plt.title('NDVI')

In [ ]:
img_ir.shape,img_r.shape


In [ ]:
neg_ndvi = ndvi*ndvi<0
plt.imshow(ndvi, cmap='RdYlGn')  # Use appropriate colormap for NDVI
plt.colorbar()
plt.title('NDVI')
# plt.savefig('/home/ajai-krishna/work/Phenocam_d3/static/images/test_ndvi.png',dpi=600)
plt.show()
# Image.fromarray(ndvi).save("/home/ajai-krishna/work/Phenocam_d3/static/images/test_ndvi.tif")
ndvi.shape

In [ ]:
np.sum(ndvi== 1)

In [ ]:
mask = '/home/ajai-krishna/work/Phenocam_d3/Mask/mask.jpg'
mask_img = Image.open(mask)

# Convert mask to grayscale ('L' mode) if it isn't already
mask_img = mask_img.convert('L')

# Resize mask to match image dimensions if needed
if mask_img.size != img_ir.size:
    mask_img = mask_img.resize(img_ir.size)

# Apply mask to IR image
img_cropped_ir = Image.composite(
    img_ir,
    Image.new('L', img_ir.size, 0),
    mask_img
)

# Apply mask to RGB image
# Resize mask for color image if dimensions differ
mask_img_color = mask_img
if mask_img.size != img_color.size:
    mask_img_color = mask_img.resize(img_color.size)

img_cropped_color = Image.composite(
    img_color,
    Image.new('RGB', img_color.size, (0, 0, 0)),
    mask_img_color
)

img_cropped_color.show()

In [ ]:
img_array_ir=np.array(img_cropped_ir)
img_array_color=np.array(img_cropped_color)
img_array_color.shape

In [ ]:
plt.imshow(img_array_ir)
plt.imshow(img_array_color[:,:,0],cmap='Reds')


In [ ]:

masked_array_color = ma.masked_where(img_array_color == 0, img_array_color)
masked_array_ir=ma.masked_where(img_array_ir==0,img_array_ir)
plt.imshow(masked_array_color[:,:,0])

In [ ]:
ir = masked_array_ir.astype(float)
r = masked_array_color[:, :, 0].astype(float)
ndvi = (ir - r) / (ir + r)
ndvi = ma.masked_invalid(ndvi)
plt.imshow(ndvi)
plt.colorbar()

In [ ]:
print("IR mean:", ir.mean())
print("Red mean:", r.mean())
print("IR > Red pixels:", np.sum(ir > r), "out of", ir.size)

In [ ]:
total = ma.mean(ndvi.flatten())  # Sum of non-masked values
total

In [ ]:
masked_array_ir.shape